In [1]:
from openai import OpenAI
import openai
import os
# API KEY 정보를 불러옵니다
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from openai import OpenAI
 
client = OpenAI()
 
assistant = client.beta.assistants.create(
  name="AI Expert",
  instructions="You are an AI expert who can answer any questions about AI.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

In [3]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="AI paper repository")

In [4]:
# Ready the files for upload to OpenAI
file_paths = ["E:\\course\\04.OpenAI\\data\\llm-critics-help-catch-llm-bugs-paper.pdf"]
for path in file_paths:
    if os.path.exists(path):
        print(f"File exists: {path}")
        print(f"File size: {os.path.getsize(path)} bytes")
    else:
        print(f"File does not exist: {path}")
file_streams = [open(path, "rb") for path in file_paths]

File exists: E:\course\04.OpenAI\data\llm-critics-help-catch-llm-bugs-paper.pdf
File size: 1230967 bytes


In [5]:

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
try:
    file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
        vector_store_id=vector_store.id, 
        files=file_streams
    )
except Exception as e:
    print(f"Error details: {e}")



In [6]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [7]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("E:\\course\\04.OpenAI\\data\\llm-critics-help-catch-llm-bugs-paper.pdf", "rb"), purpose="assistants"
)

question = "최근에 OpenAI에서 개발한 AI 모델은 무엇인가요? 상세히 한국어로 설명해주세요."
 
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": question,
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)
 
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_IBxcVEWSPFSrdKpvRPhq0ZZt'])


In [8]:
# Use the create and poll SDK helper to create a run and poll the status of
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

최근 OpenAI에서 개발한 AI 모델 중 하나는 CriticGPT입니다. 이 모델의 주요 목적은 코드 작성과 관련된 오류를 찾아내는 데 도움을 주는 것입니다. CriticGPT는 인간 피드백을 활용한 강화 학습(RLHF, Reinforcement Learning from Human Feedback)을 통해 훈련된 대형 언어 모델입니다.

### 모델 개요
CriticGPT는 GPT-4 패밀리의 Transformer 언어 모델을 기반으로 합니다. 이 모델은 자연어 피드백을 작성하여 코드의 문제점을 강조하는 방법으로 훈련됩니다. CriticGPT는 인간 평가자를 도와 모델이 작성한 코드의 오류를 더 정확하게 평가하기 위해 개발되었습니다  .

### 성능 및 특징
1. **오류 탐지 성능**:
   CriticGPT는 인간 평가자보다 코드에서 발생하는 오류를 더 효과적으로 찾아냅니다. 실제 코드에서 발생하는 오류가 포함된 코드에 대해 모델이 작성한 비평은 63%의 경우에서 인간 평가자가 작성한 비평보다 선호되었습니다[0].

2. **비교 우위**:
   CriticGPT는 ChatGPT와 비교하여 더 높은 정밀도와 재현율을 보여주며, 코드에서 인간이 삽입한 오류와 검출된 오류 모두에서 더 나은 성능을 나타냈습니다. 예를 들어, CriticGPT (RL only) 버전은 Human Inserted Bugs 분포에서 더 나은 성능을 보이며, Human Detected Bugs 분포에서는 ChatGPT보다 더 포괄적인 비평을 제공합니다 [1][2].

3. **비경계 요소 감지 성능**:
   CriticGPT와 같은 LLM(대규모 언어 모델)은 예상치 못한 오류를 감지하는 데도 유용하며, 특히 ChatGPT 훈련 데이터에서 "완벽한(flaws)"으로 분류된 여러 오류를 성공적으로 식별할 수 있습니다. 이는 CriticGPT가 코드 이외의 일반 보조 업무에서도 유용하게 사용될 수 있음을 시사합니다[3].

4. **FSBS(Force Sampling Beam Sea